In [1]:
from encoded_client.encoded import ENCODED
import pdb
import pandas as pd
import glob
import pysam
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
config_file = 'snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [3]:
ab = '../'+expand(config['data']['ab'], species='human')[0]
filt_ab = expand(config['data']['filt_ab'], species='human')[0]
gtf = expand(config['data']['cerb_gtf'], species='human')[0]
read_annot = '../'+expand(config['data']['read_annot'], species='human')[0]
t_metadata = '../'+expand(config['ref']['cerberus']['t_info'], species='human')[0]
lib_meta = '../'+expand(config['data']['meta'], species='human')[0]
swan_file = '../'+expand(config['data']['sg'], species='human')[0]
cerberus_h5 = expand(config['data']['cerb_annot'], species='human')[0]
cerb_t_metadata = '../'+expand(config['data']['t_info'], species='human')[0]
major_isos = '../'+expand(config['data']['major_isos'], species='human', obs_col='sample')[0]
pp_summary = '../'+expand(config['data']['p_pred']['summary'], species='human')[0]
ref_t_metadata = '../'+expand(config['ref']['t_info'], species='human')[0]
ref_g_metadata = '../'+expand(config['ref']['g_info'], species='human')[0]

ver = 'v40_cerberus'
min_tpm = 1
gene_subset = 'protein_coding'
obs_col = 'sample'
go_gene_subset = 'protein_coding'

m_lib_meta = '../'+expand(config['data']['meta'], species='mouse')[0]

## 231012 color for new things (10d HC and disease AD brain)

In [5]:
c_dict, order = get_tissue_age_colors()

In [6]:
c_dict['hippocampus_10d']

'#edccd2'

## 231009 adding pertinent information to the filtered biggenepred or w/e

In [136]:
sample = 'huvec'
bgp = expand(config['data']['tracks']['sample']['bgp_sort_filt'], species='human', sample='huvec')[0]
filt_ab = expand(config['data']['filt_ab'], species='human')[0]
swan_file = expand(config['data']['sg'], species='human')[0]
meta_file = expand(config['data']['meta'], species='human')[0]
ppred_file = expand(config['data']['p_pred']['summary'], species='human')[0]

In [75]:
df = pd.read_csv(bgp, sep='\t', header=None)
df.rename({3:'transcript_id'}, axis=1, inplace=True)

In [76]:
# get transcript name of each thing / novelty / novelty of each of three pieces?
ca = cerberus.read(cerberus_h5)
t_map = ca.t_map.copy(deep=True)

tss = ca.tss.copy(deep=True)
tss.rename({'Name': 'tss_id', 
            'novelty': 'tss_novelty'}, 
             axis=1, inplace=True)
tss = tss[['tss_id', 'tss_novelty']]

tes = ca.tes.copy(deep=True)
tes.rename({'Name': 'tes_id',
            'novelty': 'tes_novelty'}, 
           axis=1, inplace=True)
tes = tes[['tes_id', 'tes_novelty']]

ic = ca.ic.copy(deep=True)
ic.rename({'Name': 'ic_id', 
           'novelty': 'ic_novelty'}, 
          axis=1, inplace=True)
ic = ic[['ic_id', 'ic_novelty']]

t_map = t_map.merge(tss, how='left', on='tss_id')
t_map = t_map.merge(tes, how='left', on='tes_id')
t_map = t_map.merge(ic, how='left', on='ic_id')

t_map = t_map[['transcript_id', 'gene_name', 'transcript_name',
               'tss_id', 'ic_id', 'tes_id', 
               'tss_novelty', 'ic_novelty', 'tes_novelty']]
t_map = t_map.drop_duplicates()

In [77]:
df = df.merge(t_map,
              how='left',
              on='transcript_id')

In [78]:
# df.head()

In [79]:
# add mane info
t_df, _, _ = get_gtf_info(how='iso', ver=ver, add_stable_gid=True)
df['gid_stable'] = cerberus.get_stable_gid(df, 18)
t_df.rename({'tid':'transcript_id'}, axis=1, inplace=True)
df = df.merge(t_df[['MANE_Select', 'MANE_Plus_Clinical', 'transcript_id']],
              how='left',
              on='transcript_id')
df[['MANE_Select', 'MANE_Plus_Clinical']] = df[['MANE_Select', 'MANE_Plus_Clinical']].fillna(False)

In [80]:
# t_df.loc[t_df.gid_stable=='ENSG00000228794']

In [81]:
# df[['ic_novelty', 'MANE_Select', 'MANE_Plus_Clinical', 'transcript_id']].groupby(['ic_novelty', 'MANE_Select', 'MANE_Plus_Clinical'], dropna=False).count()
# df.loc[(df.ic_novelty=='Known')&(df.MANE_Select.isnull())]
# i think the guys w/ null mane values are likely ic matches but don't match the full length thing

In [82]:
# add gene biotype from gencode
gene_df, _, _ = get_gtf_info(how='gene', ver=ver, add_stable_gid=True)
df = df.merge(gene_df[['biotype_category', 'biotype', 'gid_stable']],
              how='left',
              on='gid_stable')

In [83]:
df.head()

,0,1,2,transcript_id,4,5,6,7,8,9,...,ic_id,tes_id,tss_novelty,ic_novelty,tes_novelty,gid_stable,MANE_Select,MANE_Plus_Clinical,biotype_category,biotype
0,chr1,14353,29408,"ENSG00000227232[2,5,1]",0,-,29408,29408,0,11,...,ENSG00000227232_5,ENSG00000227232_1,Novel,NNC,Known,ENSG00000227232,False,False,pseudogene,unprocessed_pseudogene
1,chr1,14353,29620,"ENSG00000227232[1,2,1]",0,-,29620,29620,0,9,...,ENSG00000227232_2,ENSG00000227232_1,Known,NNC,Known,ENSG00000227232,False,False,pseudogene,unprocessed_pseudogene
2,chr1,184823,199925,"ENSG00000279457[2,6,2]",0,-,199925,199925,0,11,...,ENSG00000279457_6,ENSG00000279457_2,Novel,NNC,Novel,ENSG00000279457,False,False,pseudogene,unprocessed_pseudogene
3,chr1,184823,199925,"ENSG00000279457[2,9,2]",0,-,199925,199925,0,11,...,ENSG00000279457_9,ENSG00000279457_2,Novel,NNC,Novel,ENSG00000279457,False,False,pseudogene,unprocessed_pseudogene
4,chr1,185166,195461,"ENSG00000279457[1,1,1]",0,-,195461,195461,0,10,...,ENSG00000279457_1,ENSG00000279457_1,Known,Known,Known,ENSG00000279457,False,False,pseudogene,unprocessed_pseudogene


In [84]:
# add tpm
min_tpm = 1
tpm_df = pd.read_csv(filt_ab, sep='\t')
# df = pd.read_csv(ab, sep='\t')
tpm_df, tids = get_tpm_table(tpm_df,
                   groupby='sample',
                   how='iso',
                   min_tpm=min_tpm)

Calculating iso TPM values


/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2175: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 231136
# isos >= 1 tpm: 210239
Averaging over biosample
Found 50 total samples
Number of isos reported: 210239


In [85]:
tpm_df = tpm_df[[sample]]
tpm_df.reset_index(inplace=True)
tpm_df.rename({'index':'transcript_id',
               sample: 'avg_tpm'}, axis=1, inplace=True)
df = df.merge(tpm_df, how='left', on='transcript_id')

In [86]:
# df.head()

In [87]:
# import seaborn as sns
# df = df.sort_values(by='avg_tpm', ascending=True)
# df['rank'] = [i for i in range(len(df.index))]
# sns.scatterplot(df, y='avg_tpm', x='rank')
# df.loc[df.tpm<1]

In [122]:
# pi value w/i gene

In [88]:
df_back = df.copy(deep=True)

In [69]:
sg = swan.read(swan_file)

Read in graph from data/human/swan_graph.p


In [70]:
pi_df, _ = swan.calc_pi(sg.adata, sg.t_df, obs_col='sample')

In [73]:
pi_df = df.sparse.to_dense()

In [91]:
pi_df = pi_df.transpose()

In [93]:
pi_df = pi_df[[sample]]
pi_df.reset_index(inplace=True)
pi_df.rename({'tid':'transcript_id',
              sample: 'percent_isoform'}, axis=1, inplace=True)

In [95]:
pi_df['percent_isoform'] = pi_df['percent_isoform'].fillna(0)

In [96]:
df = df.merge(pi_df, how='left', on='transcript_id')

In [ ]:
# calculate transcript length based on what's already in there

In [120]:
lens = df[['transcript_id', 10]]
lens['exon_size'] = df[10].str.split(',')
lens = lens[['transcript_id', 'exon_size']].explode('exon_size')
lens = lens.loc[lens.exon_size!='']
lens['exon_size'] = lens['exon_size'].astype(int)
lens = lens.groupby('transcript_id').sum().reset_index().rename({'exon_size':'transcript_length'}, axis=1)

df = df.merge(lens, how='left', on='transcript_id')
df.head()

,0,1,2,transcript_id,4,5,6,7,8,9,...,ic_novelty,tes_novelty,gid_stable,MANE_Select,MANE_Plus_Clinical,biotype_category,biotype,avg_tpm,percent_isoform,transcript_length
0,chr1,14353,29408,"ENSG00000227232[2,5,1]",0,-,29408,29408,0,11,...,NNC,Known,ENSG00000227232,False,False,pseudogene,unprocessed_pseudogene,0.569439,25.000000,1815
1,chr1,14353,29620,"ENSG00000227232[1,2,1]",0,-,29620,29620,0,9,...,NNC,Known,ENSG00000227232,False,False,pseudogene,unprocessed_pseudogene,0.854784,37.500000,2165
2,chr1,184823,199925,"ENSG00000279457[2,6,2]",0,-,199925,199925,0,11,...,NNC,Novel,ENSG00000279457,False,False,pseudogene,unprocessed_pseudogene,1.140129,2.985075,1867
3,chr1,184823,199925,"ENSG00000279457[2,9,2]",0,-,199925,199925,0,11,...,NNC,Novel,ENSG00000279457,False,False,pseudogene,unprocessed_pseudogene,29.923062,78.358208,1884
4,chr1,185166,195461,"ENSG00000279457[1,1,1]",0,-,195461,195461,0,10,...,Known,Known,ENSG00000279457,False,False,pseudogene,unprocessed_pseudogene,5.987865,15.671642,1497


In [118]:
# # trying to figure out why I had empty strings; it's because of that last weird comma at 
# # the end of each string from ucsc. I will filter those entries out
# lens.loc[lens.exon_size=='']
# lens.loc[lens.transcript_id=='ENSG00000227232[2,5,1]']
# df.loc[df.transcript_id=='ENSG00000227232[2,5,1]']
# ca.ic.loc[ca.ic.Name=='ENSG00000227232_5']

In [121]:
# add ENCODE experiment ids that belong to each sample, and I guess the sample name?

In [134]:
meta_df = pd.read_csv(meta_file, sep='\t')
meta_df = meta_df.loc[meta_df['sample'] == sample]
meta_df = meta_df[['ENCODE_experiment_id', 'sample']].drop_duplicates()
meta_df = meta_df.groupby('sample').agg(','.join).reset_index().rename({'ENCODE_experiment_id':'ENCODE_experiment_ids'}, axis=1)

assert len(meta_df.index) == 1

In [135]:
ids = meta_df['ENCODE_experiment_ids'].tolist()[0]
samples = meta_df['sample'].tolist()[0]
df['ENCODE_experiment_ids'] = ids
df['samples'] = samples

In [ ]:
# also add predicted CDS start / ends

In [139]:
p_df = pd.read_csv(ppred_file, sep='\t')
keep_cols = ['tid', 'CDS_Start', 'CDS_Stop', 'pid', 'nmd', 'len']
rename_d = {'tid': 'transcript_id', 'pid': 'blastp_protein_id',
            'nmd': 'predicted_nmd', 'len': 'predicted_protein_len'}
p_df = p_df[keep_cols].rename(rename_d, axis=1)
p_df.head()

,transcript_id,CDS_Start,CDS_Stop,blastp_protein_id,predicted_nmd,predicted_protein_len
0,"ENSG00000000460[3,10,3]",169795102,169852984,ENSP00000286031.6,False,911.0
1,"ENSG00000000460[1,10,3]",169795102,169852984,ENSP00000286031.6,False,911.0
2,"ENSG00000000460[1,10,6]",169795102,169852984,ENSP00000286031.6,False,911.0
3,"ENSG00000000460[1,10,1]",169795102,169852984,ENSP00000286031.6,False,911.0
4,"ENSG00000000460[3,11,3]",169802641,169852984,ENSP00000389257.3,False,824.0


In [140]:
df = df.merge(p_df, how='left', on='transcript_id')
df.head()

,0,1,2,transcript_id,4,5,6,7,8,9,...,avg_tpm,percent_isoform,transcript_length,ENCODE_experiment_ids,samples,CDS_Start,CDS_Stop,blastp_protein_id,predicted_nmd,predicted_protein_len
0,chr1,14353,29408,"ENSG00000227232[2,5,1]",0,-,29408,29408,0,11,...,0.569439,25.000000,1815,ENCSR687RWI,huvec,16747,17310,none,True,98.0
1,chr1,14353,29620,"ENSG00000227232[1,2,1]",0,-,29620,29620,0,9,...,0.854784,37.500000,2165,ENCSR687RWI,huvec,17735,24886,ENSP00000512442.1,True,101.0
2,chr1,184823,199925,"ENSG00000279457[2,6,2]",0,-,199925,199925,0,11,...,1.140129,2.985075,1867,ENCSR687RWI,huvec,187800,188538,none,True,109.0
3,chr1,184823,199925,"ENSG00000279457[2,9,2]",0,-,199925,199925,0,11,...,29.923062,78.358208,1884,ENCSR687RWI,huvec,188259,195411,ENSP00000512441.1,True,138.0
4,chr1,185166,195461,"ENSG00000279457[1,1,1]",0,-,195461,195461,0,10,...,5.987865,15.671642,1497,ENCSR687RWI,huvec,188259,195438,ENSP00000512441.1,True,147.0


In [ ]:
# now reorganize columns

In [145]:
# make columns 6 and 7 the CDS starts and remove those columns
assert len(df.loc[df[7]<df[6]]) == 0
df[6] = df[['CDS_Start', 'CDS_Stop']].min(axis=1)
df[7] = df[['CDS_Start', 'CDS_Stop']].max(axis=1)
assert len(df.loc[df[7]<df[6]]) == 0
df.drop(['CDS_Start', 'CDS_Stop'], axis=1, inplace=True)

In [ ]:
# add ENCODE annotation link (ie to the human and mouse analysis overviews) ? 

## 231004 making a GTF for each sample

In [8]:
df = pd.read_csv(filt_ab, sep='\t')
df = get_det_table(df,
                   groupby=obs_col,
                   how='iso',
                   min_tpm=min_tpm)

Calculating iso TPM values


/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2175: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 231136
# isos >= 1 tpm: 210239
Number of isos reported: 210239
Found 50 total samples


In [10]:
df = df.transpose()

In [13]:
print(len(df.index))
for c in df.columns:
    tids = df.loc[df[c]==True].index.tolist()
    # print(len(tids))
    # print(tids[:5])
    # break
    

210239
56341
['ENSG00000279457[1,1,1]', 'ENSG00000279457[2,4,2]', 'ENSG00000279457[2,6,2]', 'ENSG00000279457[2,9,2]', 'ENSG00000225880[1,1,1]']


In [15]:
gtf_df = pr.read_gtf(gtf, rename_attr=True).as_df()

Found attributes with reserved names: ['source'].
Renaming attributes with suffix '_attr'


In [17]:
gids = gtf_df.loc[gtf_df.transcript_id.isin(tids), 'gene_id'].tolist()

In [19]:
# gids[:5]

In [20]:
gtf_df = gtf_df.loc[gtf_df.transcript_id.isin(tids)]

In [21]:
gtf_df = pr.PyRanges(gtf_df)

In [22]:
gtf_df.to_gtf('test_sample_gtf.gtf')

## snakemake debug

In [5]:
species = ['human', 'mouse']
meta_df = get_meta_df(config, species)

In [7]:
meta_df.loc[meta_df.dataset.str.contains('adrenal_36d_m')]

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,document_labels,platform,RIN,spikeins,species,age,adult,sex,genotype,matching_human_samples
19,ENCSR242UEB,adrenal_36d_m_2,adrenal_36d,Adrenal gland PND36,adrenal gland,adrenal gland,NaN,tissue,#c87961,NaN,...,"'PacBio libraries v3 (October, 2020) Protocol ...",Pacific Biosciences Sequel II,6.4,True,mouse,36d,True,m,b6/cast,['adrenal gland']
20,ENCSR242UEB,adrenal_36d_m_1,adrenal_36d,Adrenal gland PND36,adrenal gland,adrenal gland,NaN,tissue,#c87961,NaN,...,"'PacBio libraries v3 (October, 2020) Protocol ...",Pacific Biosciences Sequel II,6.4,True,mouse,36d,True,m,b6/cast,['adrenal gland']


In [ ]:
# 230722 test get_lr_ids

In [13]:
def get_lr_exp_meta():
    def get_genome_bam(experiment_id):
        experiment = server.get_json(experiment_id)

        metadata = []
        default_analysis = experiment["default_analysis"]
        for f in experiment["files"]:
            analyses = [x["@id"] for x in f["analyses"]]
            # this check only applies to processed data not reads (default_analysis in analyses and)
            if f["output_type"] in ("reads", "unfiltered alignments"):
                if f["output_type"] == "unfiltered alignments" and not default_analysis in analyses:
                    # skip alignments for older analyses.
                    continue
                reps = f["technical_replicates"]
                assert len(reps) == 1
                reps = reps[0]

                extension = {
                    "fastq": ".fastq.gz",
                    "bam": ".bam",
                }[f["file_format"]]
    
                metadata.append({
                    "experiment": experiment["accession"],
                    "description": experiment["description"],
                    "simple_biosample_summary": f["simple_biosample_summary"],
                    "file": f["accession"],
                    "output_type": f["output_type"],
                    "file_size": f["file_size"],
                    "bio_rep": f["biological_replicates"][0],
                    "tech_rep": f["technical_replicates"][0]
                })

        return metadata

    server = ENCODED("www.encodeproject.org") 
    cart_url = "https://www.encodeproject.org/carts/829d339c-913c-4773-8001-80130796a367/"    
    
    cart = server.get_json(cart_url)

    metadata = []
    for experiment_id in cart["elements"]:
        metadata.extend(get_genome_bam(experiment_id))
        
    metadata = pd.DataFrame(metadata)
    metadata['name'] = metadata['experiment']+'_'+metadata['tech_rep']
    return metadata    

In [5]:
def get_lr_fastq_ids(file_format='bam'):
    metadata = get_lr_exp_meta()
    if file_format == 'bam':
        metadata = metadata.loc[metadata.output_type=='unfiltered alignments']
    elif file_format == 'fastq':
        metadata = metadata.loc[metadata.output_type=='reads']
    
    return metadata.file.tolist()

In [6]:
ids = get_lr_fastq_ids(file_format='bam')

In [7]:
len(ids)

138

In [25]:
ids = get_lr_fastq_ids(file_format='fastq')

In [26]:
len(ids)

138

In [33]:
def score_aligned_reads(filename):
    with pysam.AlignmentFile(filename, "rb") as inbam:
        aligned_reads = 0
        aligned_query_len = []
        aligned_positions = []

        for read in inbam:
            if not read.is_unmapped:
                aligned_reads += 1
                aligned_query_len.append(read.query_length)
                aligned_positions.append(len(read.positions))

        return {
            "aligned_reads": len(aligned_query_len),
            "query_len_median": np.median(aligned_query_len),
            "query_len": aligned_query_len,
            "positions_median": np.median(aligned_positions),
            "positions_len": aligned_positions,
        }
def score_fastq_reads(filename):
    read_len = []
    with xopen(filename, "rt") as instream:
        for record in SeqIO.parse(instream, "fastq"):
            read_len.append(len(record))
    
    return {
        "raw_reads": len(read_len),
        "read_len_median": np.median(read_len),
        "read_len": read_len,
    }

In [34]:
metadata = get_lr_exp_meta()
metadata.head()

,experiment,description,simple_biosample_summary,file,output_type,file_size,bio_rep,tech_rep,name
0,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF318SKH,reads,255510907,1,1_1,ENCSR293MOX_1_1
1,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF430RUG,unfiltered alignments,362638099,1,1_1,ENCSR293MOX_1_1
2,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF211SQY,reads,387217252,1,1_1,ENCSR081NRO_1_1
3,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF967OHL,unfiltered alignments,521377683,1,1_1,ENCSR081NRO_1_1
4,ENCSR257JBF,excitatory cortical neurons,male adult (53 years),ENCFF919JFJ,reads,345607718,1,1_1,ENCSR257JBF_1_1


In [35]:
counts = {}
d = 'data/human/bam/*bam'
for f in glob.glob(d):
    encid = f.rsplit('/', maxsplit=1)[1].split('.')[0]
    temp = metadata.loc[metadata.file == encid].iloc[0]
    name = temp.name
    if temp.output_type == 'unfiltered alignments':
        result = score_aligned_reads(f)
    elif temp.output_type == 'reads':
        result = score_fastq_reads(f)
    counts.setdefault(name, {}).update(result)

OSError: no BGZF EOF marker; file may be truncated

In [ ]:
array_attributes = ['read_len', 'query_len', 'positions_len']
lengths = {}

medians = {}
for name in counts:
    for k in counts[name]:
        if k in array_attributes:
            lengths[(k, name)] = counts[name][k]
        else:
            medians.setdefault(k, {})[name] = counts[name][k]

In [ ]:
def get_lr_read_lens(bams, fastqs, out):
    
    def score_aligned_reads(filename):
        with pysam.AlignmentFile(filename, "rb") as inbam:
            aligned_reads = 0
            aligned_query_len = []
            aligned_positions = []

            for read in inbam:
                if not read.is_unmapped:
                    aligned_reads += 1
                    aligned_query_len.append(read.query_length)
                    aligned_positions.append(len(read.positions))

            return {
                "aligned_reads": len(aligned_query_len),
                "query_len_median": np.median(aligned_query_len),
                "query_len": aligned_query_len,
                "positions_median": np.median(aligned_positions),
                "positions_len": aligned_positions,
            }
    def score_fastq_reads(filename):
        read_len = []
        with xopen(filename, "rt") as instream:
            for record in SeqIO.parse(instream, "fastq"):
                read_len.append(len(record))

        return {
            "raw_reads": len(read_len),
            "read_len_median": np.median(read_len),
            "read_len": read_len,
        }
    
    counts = {}
    for f in bams+fastqs:
        encid = f.rsplit('/', maxsplit=1)[1].split('.')[0]
        temp = metadata.loc[metadata.file == encid].iloc[0]
        name = temp.name
        if temp.output_type == 'unfiltered alignments':
            result = score_aligned_reads(f)
        elif temp.output_type == 'reads':
            result = score_fastq_reads(f)
        counts.setdefault(name, {}).update(result)
        
    array_attributes = ['read_len', 'query_len', 'positions_len']
    lengths = {}

    medians = {}
    for name in counts:
        for k in counts[name]:
            if k in array_attributes:
                lengths[(k, name)] = counts[name][k]
            else:
                medians.setdefault(k, {})[name] = counts[name][k]
                
    various_counts = pd.DataFrame(medians)
    various_counts.to_csv(out, sep='\t')
